<a href="https://colab.research.google.com/github/University-of-Essex-Dept-of-Government/GV918-Week04/blob/main/Week_04_Class_Exercise_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Description

In this exercise, we will continue the analysis of UK Political Data, which I presented in the lecture. We have already conducted merging Election and Brexit referendum data. We will work with additional datasets, coming from: 

https://commonslibrary.parliament.uk/constituency-dashboard/

In particular, we will use the following data:

- Population
- Unemployment
- Housing price

# Clone a data repository

As we will use the data for several weeks, I decided to create a separate repository for the data. We can clone the reposiotory to Colab working directory to start working on the data.

In [ ]:
!git clone  https://github.com/University-of-Essex-Dept-of-Government/GV918-UK-politics-data

## Rerun the code in the lecture

In [ ]:
import pandas as pd
import numpy as np


In [ ]:

df_elec = pd.read_excel("/content/GV918-UK-politics-data/Data/general-election-results-2019.xlsx", sheet_name = 'voting-summary')

df_brexit = pd.read_csv("/content/GV918-UK-politics-data/Data/brexit-vote.csv")

df_brexit.rename({'PCON11CD':'ons_id'}, axis = 1, inplace=True)

"""## Select columns"""

df_brexit.rename({'Figure to use':'leave_pct'}, axis = 1, inplace=True)

df_brexit_sub = df_brexit[['ons_id', 'leave_pct']]

df_elec['con_pct'] = df_elec['con'] / df_elec['valid_votes']
df_elec['lab_pct'] = df_elec['lab'] / df_elec['valid_votes']

df_elec.head()

"""## Merge"""
# use "left" merge

df_merge = df_elec.merge(df_brexit_sub, how='left', on = "ons_id")


# Unemployment

What we want to get from this file is the unemployment rate (or rate of unemployment claims) in November 2019, right before the election. 

## Read the file

In [ ]:
df_unemp = pd.read_excel('/content/GV918-UK-politics-data/Data/Unemployment.xlsx', sheet_name = "Data")

## Data wrangling and merge

- For this data, what we need to do is relatively simple
- The steps are:
  - Select the rows for the nearby month (Nov 2019)
  - Keep the columns for ID vars and unemployment rate
  - Merge with the main dataset

# Housing price dataset

We conduct essentially the same. Extract the most recent data in housing price, and merge it with the main dataframe.

## Read the file

In [ ]:
df_housing = pd.read_excel("/content/GV918-UK-politics-data/Data/House-prices.xlsx", 
                           sheet_name="Constituency data table")

## Data wrangling and merge

- First let's findout the data frequency. 

- Select the rows from Sep 2019
- We use the price change at the constituency level

## Population dataset

What we would like to extract from this dataset is the percentage of elderly (over 65). We need to do some data wrangling.

## Read the dataset

In [ ]:
df_pop = pd.read_excel("/content/GV918-UK-politics-data/Data/population-by-age.xlsx", 
                       sheet_name = 'Age by year data')

## Data wrangling

- This data has so many rows, but suppose that what we want to know is the population over the age of 65.

- We will clean the data using the following steps
  - Keep rows of year over 65
  - Aggregate the percentage of the population


## Merge with the main dataset

# Check correlation

- Let's check the correlation between `con_pct`, `lab_pct`, `leave_pct` and merged variables